In [1]:
cd ..

C:\Users\User\Documents\SURF 2021\RL RNN SURF\src


We first define the input data. The input data is has the following structure: We have a group of subjects (say S1 and S2) and each subject has completed the decision-making task multiple times (multiple blocks), say each subject has completed the task two times in this example, i.e., we have two blocks of data for each subject. The data within each block is a ditonary containing three numpy arrays: 'action', 'state', 'reward'. 

'action' containig the actions taken by the subject on each trial and it should be a non-zero integer or -1. If the action is -1 it will coded by a zero vector and corresponds to no-action. Dimesionlity of 'action' is B x |T| in which |T| is the number of trials.

'state' contains the state of the environment each trial. Its dimesionlity is B x |T| x |S| in which |T| is the number of trials, and |S| is the lenght of state vector. 

'reward' contains the reward received after taking each actions. Its dimesionlity is B x |T| in which |T| is the number of trials.

For example, if subject S1 has completed 6 trials in the firt block and 4 trials in the second block and subject 2 has completed 5, 6 trials in the first and second blocks respectivly, then the data structure can look like this:

   

In [2]:
#  Data
import numpy as np
import csv
data={}
test_data = {}
i=0
curr = ''
with open('quartileData\\taskData_bdiBottom.csv', 'r' ) as theFile:
    reader = csv.reader(theFile)
    headers = next(reader, None)
    for line in reader:
        if line[70] != curr:
            if i > 0:
                block = [
                    {
                        'action': np.array([actions])-1,
                        'state': np.array([states]),
#                         'reward': np.array([rewards]),
                        'id': curr,

                        'block': 0
                    }

                ]
#                 if i<226:
                data[curr] = block
#                 else:
                test_data[curr] = block
            curr = line[70]
            i += 1
            actions = []
            states = []
            rewards = []
        actions.append(int(float(line[8])))
        states.append([int(float(line[3])),int(float(line[4]))])
        rewards.append(int(float(line[10])))

In [3]:
data

{'A10FTGEV1WR18H': [{'action': array([[1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1,
           1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1,
           1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1,
           0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1,
           1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0,
           0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
           0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
           1, 0, 0, 1, 1, 1]]),
   'state': array([[[ 2,  4],
           [ 2,  4],
           [ 2,  4],
           [ 3,  4],
           [ 2,  4],
           [ 4,  3],
           [ 4,  3],
           [ 4,  2],
           [ 2,  3],
           [ 2,  3],
           [ 4,  2],
           [ 4,  3],
           [ 2,  4],
           [ 4,  2],
           [ 2,  3],
           [ 4,  2],
           [ 4, 10],
           [10,  4],
           [10

In the above example, |A|=2 (there are two actions coded as 0 and 1), and |S|=2 (the state vector has two elements). For example, if there are three stimuli in the environment, they can be coded as [1, 0, 0], [0, 1, 0], [0, 0, 1] state vectors. In this case |S|=3.

In [4]:
from actionflow.rnn_noReward.lstm_beh import LSTMBeh
worker = LSTMBeh(a_size=2, s_size=2, n_cells=20)

c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\user\documents\surf

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


Finally, we train the model:

In [5]:
from actionflow.rnn_noReward.opt_beh import OptBEH
from actionflow.util.logger import LogFile

output_path = '../results/taskData_bdiBottom_noReward/'
with LogFile(output_path, 'run.log'):
    OptBEH.optimise(worker, output_path, data, None,
                    learning_rate=0.01,
                    global_iters=50,
                    load_model_path=None
                    )

2021-08-12 14:17:14,327 - DeepRL - DEBUG - version control: (None, None)
2021-08-12 14:17:14,329 - DeepRL - DEBUG - learning rate: 0.01
2021-08-12 14:17:14,331 - DeepRL - DEBUG - global iters: 50
2021-08-12 14:17:14,333 - DeepRL - DEBUG - training data-points: 262
2021-08-12 14:17:14,334 - DeepRL - DEBUG - test data-points: None


Instructions for updating:
Use tf.cast instead.


2021-08-12 14:17:15,791 - DeepRL - DEBUG - opt started...


INFO:tensorflow:../results/taskData_bdiBottom_noReward/model-0/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-08-12 14:17:40,437 - DeepRL - DEBUG - global iter =    0 total obj: 28467.4407
2021-08-12 14:18:03,009 - DeepRL - DEBUG - global iter =    1 total obj: 27850.6971
2021-08-12 14:18:25,670 - DeepRL - DEBUG - global iter =    2 total obj: 27358.7429
2021-08-12 14:18:48,934 - DeepRL - DEBUG - global iter =    3 total obj: 27121.7556
2021-08-12 14:19:17,824 - DeepRL - DEBUG - global iter =    4 total obj: 26979.1250
2021-08-12 14:19:46,832 - DeepRL - DEBUG - global iter =    5 total obj: 26845.3165
2021-08-12 14:20:23,489 - DeepRL - DEBUG - global iter =    6 total obj: 26620.3886
2021-08-12 14:20:54,606 - DeepRL - DEBUG - global iter =    7 total obj: 26392.3666
2021-08-12 14:21:27,039 - DeepRL - DEBUG - global iter =    8 total obj: 26222.4105
2021-08-12 14:22:01,380 - DeepRL - DEBUG - global iter =    9 total obj: 26086.9020
2021-08-12 14:22:38,798 - DeepRL - DEBUG - global iter =   10 total obj: 25993.4571
2021-08-12 14:23:14,917 - DeepRL - DEBUG - global iter =   11 total obj: 259

INFO:tensorflow:../results/taskData_bdiBottom_noReward/model-final/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


In [6]:
# from actionflow.data.data_process import DataProcess
# train_merged = DataProcess.merge_data(data)
# train_merged

And then the merged data can be used for training the model as before. The test data can also be passed to the training method, in order to test the mmodel on the training data in regular intervals. Say the test data is as follows:

and we want to test the model every 10 iterations:

In [7]:
with LogFile(output_path, 'run.log'):
    OptBEH.optimise(worker, output_path, data, test_data,
                    learning_rate=0.01,
                    global_iters=50,
                    load_model_path=None,
                    test_period=10
                    )

2021-08-12 14:44:32,418 - DeepRL - DEBUG - version control: (None, None)
2021-08-12 14:44:32,420 - DeepRL - DEBUG - learning rate: 0.01
2021-08-12 14:44:32,422 - DeepRL - DEBUG - global iters: 50
2021-08-12 14:44:32,423 - DeepRL - DEBUG - training data-points: 262
2021-08-12 14:44:32,425 - DeepRL - DEBUG - test data-points: 262
2021-08-12 14:44:35,838 - DeepRL - DEBUG - opt started...
2021-08-12 14:44:35,840 - DeepRL - DEBUG - started testing...
2021-08-12 14:47:58,466 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/taskData_bdiBottom_noReward/model-0/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-08-12 14:48:27,370 - DeepRL - DEBUG - global iter =    0 total obj: 28401.0873
2021-08-12 14:48:48,773 - DeepRL - DEBUG - global iter =    1 total obj: 27901.8701
2021-08-12 14:49:10,134 - DeepRL - DEBUG - global iter =    2 total obj: 27432.1948
2021-08-12 14:49:31,620 - DeepRL - DEBUG - global iter =    3 total obj: 27159.8768
2021-08-12 14:49:53,285 - DeepRL - DEBUG - global iter =    4 total obj: 27009.6945
2021-08-12 14:50:14,709 - DeepRL - DEBUG - global iter =    5 total obj: 26913.6034
2021-08-12 14:50:35,604 - DeepRL - DEBUG - global iter =    6 total obj: 26824.2332
2021-08-12 14:51:00,744 - DeepRL - DEBUG - global iter =    7 total obj: 26683.4940
2021-08-12 14:51:27,331 - DeepRL - DEBUG - global iter =    8 total obj: 26556.6486
2021-08-12 14:51:53,671 - DeepRL - DEBUG - global iter =    9 total obj: 26447.5485
2021-08-12 14:51:53,673 - DeepRL - DEBUG - started testing...
2021-08-12 14:55:18,044 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/taskData_bdiBottom_noReward/model-10/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-08-12 14:55:46,685 - DeepRL - DEBUG - global iter =   10 total obj: 26321.4372
2021-08-12 14:56:07,980 - DeepRL - DEBUG - global iter =   11 total obj: 26176.3604
2021-08-12 14:56:29,307 - DeepRL - DEBUG - global iter =   12 total obj: 26024.8139
2021-08-12 14:56:51,183 - DeepRL - DEBUG - global iter =   13 total obj: 25969.1655
2021-08-12 14:57:12,517 - DeepRL - DEBUG - global iter =   14 total obj: 25900.0448
2021-08-12 14:57:33,956 - DeepRL - DEBUG - global iter =   15 total obj: 25848.0500
2021-08-12 14:57:57,601 - DeepRL - DEBUG - global iter =   16 total obj: 25804.0654
2021-08-12 14:58:24,572 - DeepRL - DEBUG - global iter =   17 total obj: 25720.6896
2021-08-12 14:58:49,252 - DeepRL - DEBUG - global iter =   18 total obj: 25692.1452
2021-08-12 14:59:13,569 - DeepRL - DEBUG - global iter =   19 total obj: 25653.8294
2021-08-12 14:59:13,571 - DeepRL - DEBUG - started testing...
2021-08-12 15:03:18,805 - DeepRL - DEBUG - finished testing.


...

INFO:tensorflow:../results/taskData_bdiBottom_noReward/model-30/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-08-12 15:12:10,963 - DeepRL - DEBUG - global iter =   30 total obj: 25394.4843
2021-08-12 15:12:32,739 - DeepRL - DEBUG - global iter =   31 total obj: 25376.2222
2021-08-12 15:12:54,198 - DeepRL - DEBUG - global iter =   32 total obj: 25357.9094
2021-08-12 15:13:15,693 - DeepRL - DEBUG - global iter =   33 total obj: 25299.9057
2021-08-12 15:13:37,397 - DeepRL - DEBUG - global iter =   34 total obj: 25282.7941
2021-08-12 15:13:58,957 - DeepRL - DEBUG - global iter =   35 total obj: 25306.8248
2021-08-12 15:14:20,310 - DeepRL - DEBUG - global iter =   36 total obj: 25287.1222
2021-08-12 15:14:41,708 - DeepRL - DEBUG - global iter =   37 total obj: 25264.1882
2021-08-12 15:15:02,939 - DeepRL - DEBUG - global iter =   38 total obj: 25227.9518
2021-08-12 15:15:24,075 - DeepRL - DEBUG - global iter =   39 total obj: 25203.3011
2021-08-12 15:15:24,077 - DeepRL - DEBUG - started testing...
2021-08-12 15:20:41,326 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/taskData_bdiBottom_noReward/model-40/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-08-12 15:21:23,751 - DeepRL - DEBUG - global iter =   40 total obj: 25200.9274
2021-08-12 15:21:44,990 - DeepRL - DEBUG - global iter =   41 total obj: 25272.9607
2021-08-12 15:22:06,267 - DeepRL - DEBUG - global iter =   42 total obj: 25198.4920
2021-08-12 15:22:27,418 - DeepRL - DEBUG - global iter =   43 total obj: 25198.0138
2021-08-12 15:22:46,661 - DeepRL - DEBUG - global iter =   44 total obj: 25161.6656
2021-08-12 15:23:05,321 - DeepRL - DEBUG - global iter =   45 total obj: 25177.5729
2021-08-12 15:23:23,944 - DeepRL - DEBUG - global iter =   46 total obj: 25193.6327
2021-08-12 15:23:42,701 - DeepRL - DEBUG - global iter =   47 total obj: 25104.9980
2021-08-12 15:24:02,791 - DeepRL - DEBUG - global iter =   48 total obj: 25143.4655
2021-08-12 15:24:24,011 - DeepRL - DEBUG - global iter =   49 total obj: 25101.1814
2021-08-12 15:24:24,013 - DeepRL - DEBUG - opt finished.
2021-08-12 15:24:24,014 - DeepRL - DEBUG - started testing...
2021-08-12 15:29:38,554 - DeepRL - DEBUG 

INFO:tensorflow:../results/taskData_bdiBottom_noReward/model-final/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


In [8]:
# test_data["S248"]